# Imports

In [ ]:
import CPSO as cpso
import numpy as np
import importlib

cpso = importlib.reload (cpso)

## Initialise Swarm

In [ ]:
llim = np.repeat (-100, 5)
rlim = np.repeat (100, 5)
shape = (25, 5)
sw = cpso.PSO_Optimizer.getSwarm (shape, llim, rlim)

## Supply Objective Function

In [ ]:
def sphere (x) :
    return np.sum(np.square(x), axis=1)

mizer = sw(sphere)

## Optimize

In [ ]:
opt = mizer.optimize()[0]
print ("f" + str(opt) + " = " + str(mizer.obj(opt.reshape(1,-1))[0]))